In [1]:
import pandas as pd
import re
import contractions
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Preprocess text

In [23]:

# nltk.download(['punkt', 'punk_tab' 'stopwords', 'wordnet'])
# nltk.download('punkt_tab')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def preprocess_text(text):
    # Handle NaN values
    if pd.isna(text):
        return ""
    
    # Normalization
    text = str(text).lower()  # Ensure text is a string
    text = contractions.fix(text)  # Expand contractions
    text = re.sub(r'@\w+|http\S+', '', text)  # Remove mentions and URLs
    
    # Cleaning (preserve punctuation)
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    
    # Tokenization and filtering
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    
    # Lemmatization with POS tagging
    doc = nlp(" ".join(tokens))
    tokens = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    
    return ' '.join(tokens)

In [24]:
# Load your datasets
train_df = pd.read_csv('data/train.csv')  # Replace with your actual train file
test_df = pd.read_csv('data/test.csv')    # Replace with your actual test file

# Check for missing values
print(f"Missing values in train data: {train_df['content'].isna().sum()}")
print(f"Missing values in test data: {test_df['content'].isna().sum()}")

# Preprocess text columns using lambda
train_df['cleaned_text'] = train_df['content'].apply(lambda x: preprocess_text(x))
test_df['cleaned_text'] = test_df['content'].apply(lambda x: preprocess_text(x))

# Save preprocessed data to new CSV files
train_df[['id', 'cleaned_text', 'target']].to_csv('data/train_preprocessed.csv', index=False)
test_df[['id', 'cleaned_text', 'target']].to_csv('data/test_preprocessed.csv', index=False)

print("Preprocessing complete! Files saved as train_preprocessed.csv and test_preprocessed.csv")

Missing values in train data: 242
Missing values in test data: 25


In [5]:
train_df['target'].value_counts()

target
relationship-and-family-issues     6688
anxiety                            6652
depression                         5836
ptsd-and-trauma                    1819
suicidal-thoughts-and-self-harm    1156
Name: count, dtype: int64

In [18]:
type(train_df['content'][0])

str

In [21]:
preprocess_text(train_df['content'][2])

'hello cas fair anxiety depression work lot come work become much hard last year pretty good job waitress could cope high expectation overthought everything big give feedback know well end leave shift time wrong suppose start 11 thought 1 . freak badly answer call never go back time smoke plant cope really help give coming year work drive really anyone help 27 driving lesson expensive right want give work ago feel concerned long even get hire anywhere especially licence vicious cycle need licence car get job need job get licence car haha especially age feel optimistic able get back work gap resume centrelink work though many bill want get bit ahead life know go process manufacturing course july get forklift licence soon many job available hope would something easy would find something mostly worried manage anxiety work manage overthinke managing tell thing wrong concerned make mistake point get fire anyone something similar manage ok anyone able completely transparent workplace go type

In [22]:
train_df['content'].apply(lambda x: preprocess_text(x))

AttributeError: 'float' object has no attribute 'lower'